In [1]:
# importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [12]:
''' Scraping APSRTC bus details from RedBus website '''

# Function to setup the driver
def start_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to load a webpage
def open_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to get bus routes (links and names)
def get_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    bus_links = [route.get_attribute('href') for route in routes]
    bus_names = [route.text.strip() for route in routes]
    return bus_links, bus_names

# Function to extract bus details from a specific route page
def extract_bus_details(driver, route_url, route_name):
    bus_info = []
    try:
        driver.get(route_url)
        time.sleep(5)  # Wait for the page to load
        
        try:
            # Find and click "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load
            
            # Scroll to load all bus information
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load
            
            # Scrape bus information
            bus_names = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_types = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            depart_times = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            durations = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reach_times = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            ratings = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            prices = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_avail = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            for i in range(len(bus_names)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Bus_Name": bus_names[i].text,
                    "Bus_Type": bus_types[i].text,
                    "Departing_Time": depart_times[i].text,
                    "Duration": durations[i].text,
                    "Reaching_Time": reach_times[i].text,
                    "Star_Rating": ratings[i].text if i < len(ratings) else '0',
                    "Price": prices[i].text,
                    "Seat_Availability": seat_avail[i].text if i < len(seat_avail) else '0'
                }
                bus_info.append(bus_detail)
        except Exception as e:
            print(f"Error scraping details from {route_url}: {e}")
    except Exception as e:
        print(f"Error loading page {route_url}: {e}")
    
    return bus_info

# Function to scrape all pages and extract bus details
def scrape_all_bus_data(driver, base_url, num_pages=2):
    all_bus_data = []

    for page in range(1,num_pages+1):
        try:
            # Open the base URL and handle pagination if necessary
            open_page(driver, base_url)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            # Get bus routes on the current page
            bus_links, bus_names = get_bus_routes(driver)
            
            # Iterate over bus routes using an index
            for i in range(len(bus_links)):
                link = bus_links[i]
                name = bus_names[i]
                
                # Extract bus details for each route
                bus_details = extract_bus_details(driver, link, name)
                for bus_detail in bus_details:
                    all_bus_data.append(bus_detail)
        except Exception as e:
            print(f"Error scraping page {page}: {e}")
    
    return all_bus_data


driver = start_driver()  # Start the browser
base_url = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_A = pd.DataFrame(bus_data)
df_A  # Display the DataFrame or use it as needed

# Close the browser
driver.quit()


In [29]:
# Convert bus data to a DataFrame
df_A = pd.DataFrame(bus_data)
df_A  # Display the DataFrame of APSRTC bus details


,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),23:10,06h 25m,05:35,4.5,550,22 Seats available
1,Hyderabad to Vijayawada,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:50,05h 45m,05:35,4.4,INR 409,22 Seats available
2,Hyderabad to Vijayawada,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:59,06h 31m,06:30,4.5,INR 889,11 Seats available
3,Hyderabad to Vijayawada,AdSai Dasari Travels,Non A/C Seater / Sleeper (2+1),23:05,05h 22m,04:27,4.0,475,17 Seats available
4,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),00:20,06h 10m,06:30,4.5,400,32 Seats available
...,...,...,...,...,...,...,...,...,...
848,Hyderabad to Addanki,Sri Sahasra Tours and Travels,A/C Seater / Sleeper (2+1),21:00,07h 30m,04:30,5.0,INR 450,22 Seats available
849,Hyderabad to Addanki,Sri Krishna Travels,Non A/C Seater / Sleeper (2+1),20:00,08h 30m,04:30,5.0,INR 550,18 Seats available
850,Hyderabad to Addanki,Rajeswari Travels,A/C Seater / Sleeper (2+1),22:10,05h 30m,03:40,5.0,INR 550,31 Seats available
851,Hyderabad to Addanki,Vikram Travels,A/C Sleeper (2+1),20:30,05h 40m,02:10,0,INR 935,21 Seats available


In [30]:
#converting dataframes to csv
df_A.to_csv('APSRTC_bus_details.csv', index=False)


In [22]:
'''Scraping bus details of TSRTC from Redbus'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/tsrtc?utm_source=rtchometile") #url for TSRTC buses
                      #number of pages to scrape                  
# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_T = pd.DataFrame(bus_data)driver = start_driver()  # Start the browser

# Close the browser
driver.quit()

In [25]:
# Convert bus data to a DataFrame
df_T = pd.DataFrame(bus_data)
df_T #display the data frame of TSRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),23:10,06h 25m,05:35,4.5,550,22 Seats available
1,Hyderabad to Vijayawada,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:50,05h 45m,05:35,4.4,INR 409,22 Seats available
2,Hyderabad to Vijayawada,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:59,06h 31m,06:30,4.5,INR 889,11 Seats available
3,Hyderabad to Vijayawada,AdSai Dasari Travels,Non A/C Seater / Sleeper (2+1),23:05,05h 22m,04:27,4.0,475,17 Seats available
4,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),00:20,06h 10m,06:30,4.5,400,32 Seats available
...,...,...,...,...,...,...,...,...,...
848,Hyderabad to Addanki,Sri Sahasra Tours and Travels,A/C Seater / Sleeper (2+1),21:00,07h 30m,04:30,5.0,INR 450,22 Seats available
849,Hyderabad to Addanki,Sri Krishna Travels,Non A/C Seater / Sleeper (2+1),20:00,08h 30m,04:30,5.0,INR 550,18 Seats available
850,Hyderabad to Addanki,Rajeswari Travels,A/C Seater / Sleeper (2+1),22:10,05h 30m,03:40,5.0,INR 550,31 Seats available
851,Hyderabad to Addanki,Vikram Travels,A/C Sleeper (2+1),20:30,05h 40m,02:10,0,INR 935,21 Seats available


In [26]:
df_T.to_csv('TSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [33]:
'''Scraping bus detais of UPSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/upsrtc?utm_source=rtchometile") #url for UPSRTC buses
                

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

#Convert bus data to data frame
df_U= pd.DataFrame(bus_data)
                      
                      



In [34]:
df_U #display the data frame of UPSRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,UPSRTC - HDD0092,Ordinary Non AC Seater 2+3,18:20,05h 42m,00:02,3.3,INR 415,52 Seats available
1,Delhi to Bareilly,UPSRTC - HDD0006,Janrath AC Seater 2+2,18:30,06h 30m,01:00,3.3,INR 602,36 Seats available
2,Delhi to Bareilly,UPSRTC - HDD0113,Rajdhani Non AC Seater 2+3,18:31,06h 29m,01:00,3.3,INR 444,51 Seats available
3,Delhi to Bareilly,UPSRTC - SRG0226,Ordinary Non AC Seater 2+3,19:00,05h 19m,00:19,3.3,INR 450,52 Seats available
4,Delhi to Bareilly,UPSRTC - PLB0056,Rajdhani Non AC Seater 2+3,19:01,05h 59m,01:00,3.3,INR 454,51 Seats available
...,...,...,...,...,...,...,...,...,...
802,Kanpur (Uttar Pradesh) to Bareilly,UPSRTC - BLY0094,Janrath AC Seater 2+2,22:01,07h 00m,05:01,3.4,INR 589,36 Seats available
803,Kanpur (Uttar Pradesh) to Bareilly,UPSRTC - PLB0044,Ordinary Non AC Seater 2+3,22:45,06h 15m,05:00,3.3,INR 402,52 Seats available
804,Kanpur (Uttar Pradesh) to Bareilly,UPSRTC - BLY0096,Janrath AC Seater 2+2,23:03,06h 58m,06:01,3.3,INR 589,36 Seats available
805,Kanpur (Uttar Pradesh) to Bareilly,Samay Shatabdi Travels Pvt Ltd,Bharat Benz A/C Sleeper (2+1),20:00,06h 45m,02:45,4.2,INR 849,18 Seats available


In [35]:
df_U.to_csv('UPSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping  the bus details of South Bengal from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ('https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile') 
#url for South Bengal buses 

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_SB = pd.DataFrame(bus_data)

In [40]:
df_SB #display the data frame of South Bengal buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur to Calcutta,Greenline (Karunamoyee),Volvo A/C Seater (2+2),17:40,03h 40m,21:20,4.0,INR 486,2 Seats available
1,Durgapur to Calcutta,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),18:15,03h 45m,22:00,4.3,INR 486,9 Seats available
2,Durgapur to Calcutta,Express Line,Volvo 9600 A/C Seater (2+2),18:30,04h 20m,22:50,4.5,INR 486,12 Seats available
3,Kolkata to Burdwan,SBSTC-KOLKATA - BURDWAN - 18:15 (BWN DEPOT) - 96,Non AC Seater (2+3),18:15,02h 00m,20:15,2.9,INR 95,44 Seats available
4,Kolkata to Burdwan,SBSTC-KARUNAMOYEE - BURDWAN - 18:15 (BWN DEPOT...,Non AC Seater (2+3),18:15,02h 00m,20:15,3.1,INR 100,30 Seats available
...,...,...,...,...,...,...,...,...,...
139,Kolkata to Heria,Basu Travels,A/C Seater / Sleeper (2+2),23:59,03h 01m,03:00,2.7,INR 339,32 Seats available
140,Kolkata to Heria,Basu Travels,A/C Seater Push Back (2+3),23:55,02h 50m,02:45,2.7,INR 469,21 Seats available
141,Digha to Barasat (West Bengal),Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.6,INR 420,15 Seats available
142,Digha to Barasat (West Bengal),Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,04:30,2.1,INR 550,62 Seats available


In [41]:
df_SB.to_csv('South_Bengal_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of HRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/hrtc?utm_source=rtchometile") #url for HRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_H = pd.DataFrame(bus_data)


In [43]:
df_H #display the data frame of HRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,4.0,INR 912,12 Seats available
1,Delhi to Shimla,HRTC - 591,A/C Executive (2+3),21:00,10h 20m,07:20,3.7,INR 632,41 Seats available
2,Delhi to Shimla,HRTC - 8,Himsuta AC Seater Volvo/Scania 2+2,21:02,09h 28m,06:30,4.7,INR 912,17 Seats available
3,Delhi to Shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,2.5,INR 512,37 Seats available
4,Delhi to Shimla,HRTC - 20,Himsuta AC Seater Volvo/Scania 2+2,21:36,07h 54m,05:30,4.1,INR 887,23 Seats available
...,...,...,...,...,...,...,...,...,...
466,Delhi to Ghumarwin,HRTC - 77,Himsuta AC Seater Volvo/Scania 2+2,22:15,09h 00m,07:15,4.7,INR 982,18 Seats available
467,Delhi to Ghumarwin,Bedi Travels,Volvo A/C Semi Sleeper (2+2),21:30,07h 30m,05:00,4.8,540,20 Seats available
468,Delhi to Ghumarwin,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:45,07h 00m,04:45,4.4,683,7 Seats available
469,Delhi to Ghumarwin,Northern Travels,Volvo A/C Semi Sleeper (2+2),21:20,07h 20m,04:40,3.9,INR 450,35 Seats available


In [44]:
df_H .to_csv('HRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of WBTC from Redbus website '''
driver = start_driver()  # Start the browser
base_url=("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile") #url of WBTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_W = pd.DataFrame(bus_data)

In [46]:
df_W #display the data frame of WBTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.6,INR 420,15 Seats available
1,Digha to Barasat (West Bengal),Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,04:30,2.1,INR 550,62 Seats available
2,Digha to Barasat (West Bengal),The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,5.0,INR 650,3 Seats available
3,Durgapur to Calcutta,Express Line,Volvo 9600 A/C Seater (2+2),18:30,04h 20m,22:50,4.5,INR 486,10 Seats available
4,Durgapur to Calcutta,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),18:35,03h 45m,22:00,4.3,INR 486,6 Seats available
...,...,...,...,...,...,...,...,...,...
70,Kolkata to Asansol (West Bengal),Greenline,Volvo A/C Seater (2+2),18:35,04h 30m,23:05,4.1,INR 524,27 Seats available
71,Kolkata to Asansol (West Bengal),Royal Cruiser,Volvo Multi-Axle B9R A/c Seater (2+2),19:30,04h 45m,00:15,3.7,INR 600,21 Seats available
72,Kolkata to Asansol (West Bengal),Pariwartan Bus Service,NON A/C Seater / Sleeper (2+2),18:30,06h 00m,00:30,2.4,INR 500,54 Seats available
73,Kolkata to Asansol (West Bengal),Beauty Travels,NON A/C Seater / Sleeper (2+2),20:20,04h 10m,00:30,2.6,INR 500,32 Seats available


In [47]:
df_W .to_csv('WBTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping  bus details of  RSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile') #url of RSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_R = pd.DataFrame(bus_data)

In [49]:
df_R #display the data frame of RSRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,RSRTC - 189622,Express Non AC Seater 2+3,22:00,07h 00m,05:00,3.5,INR 294,40 Seats available
1,Udaipur to Jodhpur,Raj Travel,A/C Sleeper (2+1),22:30,07h 00m,05:30,4.4,INR 550,20 Seats available
2,Udaipur to Jodhpur,Jain travels regd,A/C Seater / Sleeper (2+1),22:15,06h 15m,04:30,4.1,INR 460,15 Seats available
3,Udaipur to Jodhpur,Kalpana Travels,NON AC Seater / Sleeper 2+1,22:00,06h 54m,04:54,4.1,INR 300,24 Seats available
4,Udaipur to Jodhpur,Jain travels regd,NON AC Seater / Sleeper 2+1,22:15,06h 15m,04:30,4.3,INR 360,19 Seats available
...,...,...,...,...,...,...,...,...,...
548,Jaipur (Rajasthan) to Kota(Rajasthan),Navrang Sanjay Travels,A/C Seater / Sleeper (2+1),22:30,02h 00m,00:30,2.7,INR 476.19,34 Seats available
549,Jaipur (Rajasthan) to Kota(Rajasthan),Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,05h 30m,04:30,2.3,INR 400,31 Seats available
550,Jaipur (Rajasthan) to Kota(Rajasthan),New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),20:15,04h 50m,01:05,1.0,INR 599,38 Seats available
551,Jaipur (Rajasthan) to Kota(Rajasthan),Jai Shree Ganesh Travels,AC Sleeper (2+1),19:45,04h 45m,00:30,1.0,INR 2380.95,36 Seats available


In [50]:
df_R .to_csv('RSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of KSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile") #url of KSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_K = pd.DataFrame(bus_data)

In [53]:
df_K #display the data frame of KSRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,07h 45m,03:45,3.9,INR 473,8 Seats available
1,Bangalore to Kozhikode,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:30,09h 25m,06:55,3.6,INR 640,11 Seats available
2,Bangalore to Kozhikode,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:10,09h 25m,07:35,4.1,INR 640,17 Seats available
3,Bangalore to Kozhikode,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 40m,09:40,3.2,INR 469,10 Seats available
4,Bangalore to Kozhikode,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,10h 20m,09:50,3.6,INR 469,15 Seats available
...,...,...,...,...,...,...,...,...,...
276,Kottayam to Kozhikode,KSRTC (Kerala) - 2690,Super Fast Non AC Seater (2+3),23:06,06h 24m,05:30,2.6,INR 332,37 Seats available
277,Kottayam to Kozhikode,Madhavi Travels,A/C Sleeper (2+1),19:00,08h 00m,03:00,5.0,INR 999,13 Seats available
278,Kottayam to Kozhikode,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,07h 19m,02:29,3.0,INR 750,20 Seats available
279,Kottayam to Kozhikode,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,2.5,INR 650,20 Seats available


In [54]:
df_K .to_csv('KSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of ASTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/astc') #url of ASTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_AS = pd.DataFrame(bus_data)


In [56]:
df_AS #display the data frame of ASTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.0,INR 400,11 Seats available
1,Tezpur to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:45,05h 30m,03:15,3.8,INR 550,8 Seats available
2,Guwahati to Tezpur,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,3.5,400,5 Seats available
3,Guwahati to Tezpur,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.3,INR 500,14 Seats available
4,Guwahati to Tezpur,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:15,04h 30m,01:45,3.2,INR 450,12 Seats available
5,Guwahati to Tezpur,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:45,04h 15m,02:00,3.0,INR 450,13 Seats available
6,Guwahati to Tezpur,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),20:00,05h 15m,01:15,2.9,400,16 Seats available
7,Guwahati to Tezpur,Himalayan Travels,NON AC Seater / Sleeper 2+1,20:30,06h 00m,02:30,1.0,INR 500,12 Seats available
8,Guwahati to Tezpur,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,4.1,INR 600,12 Seats available
9,Guwahati to Tezpur,RAYAN TRAVELS,Non A/C Seater / Sleeper (2+1),19:15,04h 44m,23:59,4.0,550,27 Seats available


In [57]:
df_AS.to_csv("ASTC_bus_detais.csv",index=False)  #converting dataframes to csv

In [ ]:
'''Scraping Bus details of BSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile') #url of BSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_BS = pd.DataFrame(bus_data)

In [60]:
df_BS #display the data frame of BSRTC buses

,Route_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.4,585,25 Seats available
1,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,1.9,630,6 Seats available
2,Patna (Bihar) to Bettiah,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,2.5,INR 350,51 Seats available
3,Patna (Bihar) to Motihari,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),21:20,03h 40m,01:00,3.1,INR 230,17 Seats available
4,Patna (Bihar) to Motihari,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:00,06h 00m,04:00,4.5,INR 550,40 Seats available
...,...,...,...,...,...,...,...,...,...
153,Patna (Bihar) to Saharsa,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),21:59,08h 01m,06:00,2.8,INR 400,8 Seats available
154,Patna (Bihar) to Saharsa,Rukmanti (AT),A/C Seater / Sleeper (2+1),20:00,09h 00m,05:00,2.2,509,27 Seats available
155,Patna (Bihar) to Saharsa,Tiwari Motors,A/C Seater / Sleeper (2+1),20:00,09h 05m,05:05,2.0,INR 449,28 Seats available
156,Patna (Bihar) to Saharsa,Raj Rath,A/C Seater / Sleeper (2+2),21:00,09h 00m,06:00,2.6,INR 425,24 Seats available


In [61]:
df_BS .to_csv('BSRTC_bus_detais.csv',index=False) #converting dataframes to csv